In [1]:
from datetime import datetime, timedelta
import json
from pymlb_statsapi import api


In [2]:
def jprint(obj: dict | list) -> None:
    print(json.dumps(obj, indent=2))


def getTeamId(teamName: str, season: int = None) -> int:
    seasonYear = season or datetime.now().year
    response = api.Team.teams(sportId=1, season=seasonYear)
    teamsData = response.json().get("teams", [])
    teamLookup = []
    for team in teamsData:
        namesList = [
            team.get("name").lower(),
            team.get("teamName").lower(),
            team.get("abbreviation").lower(),
            team.get("shortName").lower(),
            team.get("locationName").lower(),
        ]
        teamLookup.append(
            {
                "id": team.get("id"),
                "name": team.get("name"),
                "namesList": namesList,
            }
        )
    for team in teamLookup:
        if teamName.lower() in team.get("namesList", []):
            return team.get("id")
    return None


In [3]:
def getTeamRoster(teamName: str, season: int = None) -> list:
    teamId = getTeamId(teamName, season)
    if not teamId:
        return []
    year = season or datetime.now().year
    response = api.Team.roster(teamId=teamId, season=year, rosterType="active").json()
    rosterData = response.get("roster", [])
    return rosterData


In [4]:
jprint(getTeamRoster("Braves"))

[
  {
    "person": {
      "id": 700363,
      "fullName": "AJ Smith-Shawver",
      "link": "/api/v1/people/700363"
    },
    "jerseyNumber": "32",
    "position": {
      "code": "1",
      "name": "Pitcher",
      "type": "Pitcher",
      "abbreviation": "P"
    },
    "status": {
      "code": "A",
      "description": "Active"
    },
    "parentTeamId": 144
  },
  {
    "person": {
      "id": 607481,
      "fullName": "Aaron Bummer",
      "link": "/api/v1/people/607481"
    },
    "jerseyNumber": "49",
    "position": {
      "code": "1",
      "name": "Pitcher",
      "type": "Pitcher",
      "abbreviation": "P"
    },
    "status": {
      "code": "A",
      "description": "Active"
    },
    "parentTeamId": 144
  },
  {
    "person": {
      "id": 663586,
      "fullName": "Austin Riley",
      "link": "/api/v1/people/663586"
    },
    "jerseyNumber": "27",
    "position": {
      "code": "5",
      "name": "Third Base",
      "type": "Infielder",
      "abbreviation": "3B

In [30]:
def printTeamRoster(teamName: str, season: int = None) -> None:
    roster = getTeamRoster(teamName, season)
    teamRoster = [
        {
            "name": player.get("person", {}).get("fullName"),
            "jerseyNumber": player.get("jerseyNumber"),
            "position": player.get("position", {}).get("abbreviation"),
            "positionGroup": "Pitchers" if player.get("position", {}).get("abbreviation") == "P" else "Infielders" if player.get("position", {}).get("abbreviation") in ["C", "1B", "2B", "SS", "3B"] else "OutFielders" if player.get("position", {}).get("abbreviation") in ["LF", "CF", "RF"] else "Other"
        }
        for player in roster
    ]
    teamRoster = sorted(teamRoster, key=lambda x: (x["positionGroup"], int(x["jerseyNumber"]) if x["jerseyNumber"] else 999, x["name"]))

    maxNumberLength = len(max([player.get("jerseyNumber") for player in teamRoster], key=len))
    maxNameLength = len(max([player.get("name") for player in teamRoster], key=len))
    maxPositionLength = len(max([player.get("position") for player in teamRoster], key=len))

    header = f" # | {'Player':<{maxNameLength}} | {'Pos':<{maxPositionLength}}"

    print(header)
    # print("-" * len(header))
    print(f"---+{'-' * (maxNameLength + 2)}+{'-' * (maxPositionLength + 2)}")
    for player in teamRoster:
        number = player.get("jerseyNumber") or "--"
        name = player.get("name")
        position = player.get("position")
        print(f"{number:>{maxNumberLength}} | {name:<{maxNameLength}} | {position:<{maxPositionLength}}")


In [31]:
printTeamRoster("Braves")

 # | Player                | Pos
---+-----------------------+----
 1 | Ozzie Albies          | 2B
 7 | Ha-Seong Kim          | SS
12 | Sean Murphy           | C 
24 | Nacho Alvarez Jr.     | SS
27 | Austin Riley          | 3B
28 | Matt Olson            | 1B
30 | Drake Baldwin         | C 
-- | Jorge Mateo           | SS
-- | Mauricio Dubón        | 2B
13 | Ronald Acuña Jr.      | RF
17 | Jurickson Profar      | LF
18 | Mike Yastrzemski      | RF
23 | Michael Harris II     | CF
36 | Eli White             | RF
26 | Raisel Iglesias       | P 
32 | AJ Smith-Shawver      | P 
33 | Joey Wentz            | P 
40 | Reynaldo López        | P 
46 | Dylan Dodd            | P 
49 | Aaron Bummer          | P 
51 | Chris Sale            | P 
52 | Dylan Lee             | P 
55 | Bryce Elder           | P 
56 | Spencer Schwellenbach | P 
62 | Daysbel Hernández     | P 
64 | Hurston Waldrep       | P 
65 | Hunter Stratton       | P 
66 | Grant Holmes          | P 
70 | Rolddy Muñoz          | P 
75 | D

In [44]:
sampleGame = {
    "gamePk": 777735,
    "gameGuid": "67e94d61-4c7e-43d4-979d-b6cf81a0af11",
    "link": "/api/v1.1/game/777735/feed/live",
    "gameType": "R",
    "season": "2025",
    "gameDate": "2025-05-29T17:05:00Z",
    "officialDate": "2025-05-29",
    "rescheduledFrom": "2025-05-28T22:45:00Z",
    "rescheduledFromDate": "2025-05-28",
    "status": {
        "abstractGameState": "Final",
        "codedGameState": "F",
        "detailedState": "Final",
        "statusCode": "F",
        "startTimeTBD": False,
        "abstractGameCode": "F",
    },
    "teams": {
        "away": {
            "team": {
                "id": 144,
                "name": "Atlanta Braves",
                "link": "/api/v1/teams/144",
            },
            "leagueRecord": {"wins": 25, "losses": 29, "pct": ".463"},
            "score": 4,
            "isWinner": False,
            "splitSquad": False,
            "seriesNumber": 18,
        },
        "home": {
            "team": {
                "id": 143,
                "name": "Philadelphia Phillies",
                "link": "/api/v1/teams/143",
            },
            "leagueRecord": {"wins": 36, "losses": 19, "pct": ".655"},
            "score": 5,
            "isWinner": True,
            "splitSquad": False,
            "seriesNumber": 18,
        },
    },
    "venue": {
        "id": 2681,
        "name": "Citizens Bank Park",
        "link": "/api/v1/venues/2681",
    },
    "content": {"link": "/api/v1/game/777735/content"},
    "isTie": False,
    "gameNumber": 1,
    "publicFacing": True,
    "doubleHeader": "S",
    "gamedayType": "P",
    "tiebreaker": "N",
    "calendarEventID": "14-777735-2025-05-29",
    "seasonDisplay": "2025",
    "dayNight": "day",
    "description": "Makeup of 5/28 PPD",
    "scheduledInnings": 9,
    "reverseHomeAwayStatus": False,
    "inningBreakLength": 145,
    "gamesInSeries": 3,
    "seriesGameNumber": 2,
    "seriesDescription": "Regular Season",
    "recordSource": "S",
    "ifNecessary": "N",
    "ifNecessaryDescription": "Normal Game",
}

gameDate = sampleGame.get("gameDate")
print(datetime.strptime(gameDate, "%Y-%m-%dT%H:%M:%SZ").strftime("%m/%d/%Y %I:%M %p"))
print(datetime.fromisoformat(gameDate).strftime("%a, %#m/%#d/%y"))
print(datetime.fromisoformat(gameDate).strftime("%c"))

05/29/2025 05:05 PM
Thu, 5/29/25
Thu May 29 17:05:00 2025


#### Examples That Return Formatted/Print Results...

In [ ]:
def printTeamTotalSeasonWins(teamName: str, season: int = None) -> None:
    teamId = getTeamId(teamName)
    year = season or datetime.now().year
    isCurrSeason = year == datetime.now().year

    response = api.Schedule.schedule(
        sportId=1, teamId=teamId, startDate=f"{year}-01-01", endDate=f"{year}-12-31"
    ).json()

    gameDates = response.get("dates", [])

    totalWins = 0
    for games in gameDates:
        for game in games.get("games", []):
            teams = game.get("teams", {})
            homeTeamResult = {
                "id": teams.get("home", {}).get("team", {}).get("id"),
                "isWinner": teams.get("home", {}).get("isWinner", False),
            }
            awayTeamResult = {
                "id": teams.get("away", {}).get("team", {}).get("id"),
                "isWinner": teams.get("away", {}).get("isWinner", False),
            }
            winningTeamId = (
                homeTeamResult["id"]
                if homeTeamResult["isWinner"]
                else awayTeamResult["id"] if awayTeamResult["isWinner"] else None
            )

            if winningTeamId == teamId:
                totalWins += 1

    print(
        f"The {teamName}{' have won ' if isCurrSeason else ' won '}{totalWins} games {'this season' if isCurrSeason else f'in {year}'}."
    )


In [ ]:
printTeamTotalSeasonWins("Braves", 2021)
printTeamTotalSeasonWins("Braves", 2023)
printTeamTotalSeasonWins("Braves", 2025)
printTeamTotalSeasonWins("Braves")


The Braves won 114 games in 2021.
The Braves won 124 games in 2023.
The Braves won 90 games in 2025.
The Braves have won 0 games this season.


#### SOME MORE COOL EXAMPLES TO TRY AND CONVERT TO USING THIS NEW API...


In [ ]:
# # Example: Print the linescores of all games won by the Atlanta Braves in July 2025
# for x in [
#     y
#     for y in statsapi.schedule(team=144, start_date="07/01/2025", end_date="07/31/2025")
#     if y.get("winning_team", "") == "Atlanta Braves"
# ]:
#     print(
#         "%s\nWinner: %s, Loser: %s\n%s\n\n"
#         % (
#             x["game_date"],
#             x["winning_team"],
#             x["losing_team"],
#             statsapi.linescore(x["game_id"]),
#         )
#     )


In [ ]:
# # Example: Print the linescores of all games won by the Atlanta Braves in July 2025
# for x in [
#     y
#     for y in statsapi.schedule(team=144, start_date="07/01/2025", end_date="07/31/2025")
#     if y.get("winning_team", "") == "Atlanta Braves"
# ]:
#     print(
#         "%s\nWinner: %s, Loser: %s\n%s\n\n"
#         % (
#             x["game_date"],
#             x["winning_team"],
#             x["losing_team"],
#             statsapi.linescore(x["game_id"]),
#         )
#     )

# # Example: Print the Braves 40-man Roster on opening day of the 2025 season
# print(
#     "Braves 40-man roster on opening day of the 2025 season:\n%s"
#     % statsapi.roster(
#         144,
#         "40Man",
#         date=statsapi.get("season", {"seasonId": 2025, "sportId": 1})["seasons"][0][
#             "regularSeasonStartDate"
#         ],
#     )
# )

# # Example: Print boxscore and linescore from Braves most recent game (which may be in progress or may not have started yet based on MLB response to 'last game' request)
# most_recent_game_id = statsapi.last_game(144)
# print(
#     statsapi.boxscore(most_recent_game_id),
#     statsapi.linescore(most_recent_game_id),
#     sep="\n\n",
# )

# # Other Examples:
# # - find team with longest name
# longest_team_name = max(
#     [
#         x["name"]
#         for x in statsapi.get(
#             "teams", {"sportIds": 1, "activeStatus": "Yes", "fields": "teams,name"}
#         )["teams"]
#     ],
#     key=len,
# )
# print(
#     "The team with the longest name is %s, at %s characters."
#     % (longest_team_name, len(longest_team_name))
# )

# # - print standings from July 4, 2025
# print(statsapi.standings(date="07/04/2025"))

# #  - print top 5 team leaders in walks for 2025 Braves
# print(statsapi.team_leaders(144, "walks", limit=5, season=2025))

# # - print top 10 all time career leaders in doubles
# print(
#     statsapi.league_leaders("doubles", statGroup="hitting", statType="career", limit=10)
# )

# # - print Ronald Acuña Jr.'s career hitting stats
# print(
#     statsapi.player_stats(
#         statsapi.lookup_player("Ronald Acuña Jr.")[0]["id"],
#         "hitting",
#         "career",
#     )
# )

# # - print list of scoring plays from 6/23/2024 Braves @ Yankees
# print(statsapi.game_scoring_plays(745727))
